In [33]:
import pandas as pd
import numpy as np

## Let's get all the required data

We need to pull in,
- option orders
- option events
- option positions

In [312]:
# 
# calculate P/L for Option Orders
# 

oos = pd.read_csv("../option_orders.csv")
oos["symbol"] = oos["chain_symbol"]
oos = oos[oos.state == "filled"]

agg_oos = ( oos[["symbol", "processed_premium"]].
                groupby(["symbol"]).
                agg({
                    "processed_premium": {
                        "oo_total": "sum",
                        "oo_count": "count"
                }}).
                sort_values(("processed_premium", "oo_total"))
            )

agg_oos.columns = [ x[1] for x in agg_oos.columns.ravel()]
agg_oos.sort_values(["symbol"])

/Users/weston/.pyenv/versions/simple_portfolio/lib/python3.6/site-packages/pandas/core/groupby/groupby.py:4656: FutureWarning: using a dict with renaming is deprecated and will be removed in a future version
  return super(DataFrameGroupBy, self).aggregate(arg, *args, **kwargs)


,oo_total,oo_count
symbol,,
AAPL,49.0,6
AMZN,58.0,5
CHK,-42.0,7
COST,-15.0,2
CRM,22.0,7
EEM,-48.0,8
F,-18.0,3
FB,-16.0,2
FIT,-78.0,3


In [279]:
# 
# caculate P/L for Option Events
# 

oes = pd.read_csv("../option_events.csv")

exercised_or_assigned_oes = oes[oes.type.str.contains("assignment|exercise", regex=True)]

agg_oes = ( exercised_or_assigned_oes[["symbol", "total_cash_amount"]].
                groupby(["symbol"]).
                agg({
                    "total_cash_amount": {
                        "oe_total": "sum",
                        "oe_count": "count"
                    }
                }).
                sort_values(("total_cash_amount", "oe_total")) )

agg_oes.columns = [ x[1] for x in agg_oes.columns.ravel()]

agg_oes

/Users/weston/.pyenv/versions/simple_portfolio/lib/python3.6/site-packages/pandas/core/groupby/groupby.py:4656: FutureWarning: using a dict with renaming is deprecated and will be removed in a future version
  return super(DataFrameGroupBy, self).aggregate(arg, *args, **kwargs)


,oe_total,oe_count
symbol,,
TLT,99.82,2


In [361]:
#
# calculate P/L for (open) Option Positions
# 

ops = pd.read_csv("../option_positions.csv")
ops["symbol"] = ops["chain_symbol"]

# log a t
ops["coef"] = np.where(ops["type"] == "long", 1.0, -1.0)

# calcalate diff between yesterday and today
ops["market_value"]      = ops["quantity"] * ops["adjusted_mark_price"] 
ops["pv_prev"]   = ops["quantity"] * ops["previous_close_price"]
ops["pv_orig"]   = ops["coef"] * ops["quantity"] * ops["average_price"] / 100.0

ops["return_today"] = ops["market_value"] - ops["pv_prev"]
ops["return_total"] = ops["market_value"] - ops["pv_orig"]


agg_ops = ( ops[["symbol", "market_value", "return_today", "pv_prev", "return_total", "pv_orig",
                    "delta", "theta"]].
               groupby(["symbol"]).
               agg({
                   "return_today": "sum", 
                   "return_total": "sum", 
                   "market_value": "sum", 
                   "pv_prev": "sum", 
                   "pv_orig": "sum", 
                   "delta": "sum",
                   "theta": "sum"}).
               sort_values(["symbol"]) )

ops.columns 

agg_ops

,return_today,return_total,market_value,pv_prev,pv_orig,delta,theta
symbol,,,,,,,
IEF,-0.14,-0.19,-0.50,-0.36,-0.31,-0.217583,0.002844
QQQ,-0.05,-0.05,-0.14,-0.09,-0.09,0.022651,0.000001
SBUX,0.06,0.03,0.36,0.30,0.33,0.061486,0.020087
SPY,2.65,2.55,3.30,0.65,0.75,-0.017967,-0.002674
SQ,-0.08,-0.08,0.49,0.57,0.57,-0.015347,0.003823
SQQQ,-0.20,-0.53,0.06,0.26,0.59,-0.133427,-0.023588
TLT,-0.37,-0.79,-2.09,-1.72,-1.30,-0.475080,-0.003241
UVXY,-0.12,0.01,0.63,0.75,0.62,-0.165845,-0.005061


In [282]:
a1 = agg_oos.index.values
a2 = agg_oes.index.values
a3 = agg_ops.index.values

symbols = np.unique(np.concatenate([a1, a2, a3], axis=None))

In [306]:
# 
# get all symbols
# 

df = pd.DataFrame()
df["symbol"] = symbols


In [307]:
# 
# join all option orders P/L
# 


df = df.join(agg_oos[["oo_total"]], on="symbol", how="left")
df = df.join(agg_oes[["oe_total"]], on="symbol", how="left")
df = df.join(agg_ops[["pv_orig_change"]], on="symbol", how="left")
df = df.fillna(0)

In [308]:
df["pv_orig_change"] = df["pv_orig_change"] * 100.0
df

,symbol,oo_total,oe_total,pv_orig_change
0,AAPL,49.0,0.00,0.0
1,AMZN,58.0,0.00,0.0
2,CHK,-42.0,0.00,0.0
3,COST,-15.0,0.00,0.0
4,CRM,22.0,0.00,0.0
5,EEM,-48.0,0.00,0.0
6,F,-18.0,0.00,0.0
7,FB,-16.0,0.00,0.0
8,FIT,-78.0,0.00,0.0
9,FXI,-14.0,0.00,0.0


In [310]:
df["total"] = df.oo_total + df.oe_total + df.pv_orig_change

In [311]:
df.sort_values(["total"])

,symbol,oo_total,oe_total,pv_orig_change,total
20,SQQQ,-61.0,0.00,-53.0,-114.00
17,SBUX,-110.0,0.00,3.0,-107.00
24,UNG,-79.0,0.00,0.0,-79.00
8,FIT,-78.0,0.00,0.0,-78.00
19,SQ,-57.0,0.00,-8.0,-65.00
15,QQQ,-56.0,0.00,-5.0,-61.00
5,EEM,-48.0,0.00,0.0,-48.00
2,CHK,-42.0,0.00,0.0,-42.00
22,TBT,-40.0,0.00,0.0,-40.00
16,RUN,-29.0,0.00,0.0,-29.00
